In [2]:
import requests
import json
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

In [3]:
test = {"City": ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"], "State": ["NY", "CA", "IL", "TX", "AZ", "PA", "TX", "CA", "TX", "CA"]}
test_df = pd.DataFrame(data=test)
test_df 

,City,State
0,New York,NY
1,Los Angeles,CA
2,Chicago,IL
3,Houston,TX
4,Phoenix,AZ
5,Philadelphia,PA
6,San Antonio,TX
7,San Diego,CA
8,Dallas,TX
9,San Jose,CA


In [3]:
test_df .replace(' ', '+',regex=True,inplace=True)
test_df

,City,State
0,New+York,NY
1,Los+Angeles,CA
2,Chicago,IL
3,Houston,TX
4,Phoenix,AZ
5,Philadelphia,PA
6,San+Antonio,TX
7,San+Diego,CA
8,Dallas,TX
9,San+Jose,CA


In [4]:
test_df['var'] = test_df.agg('{0[City]}%2C+{0[State]}'.format, axis=1)
test_df

,City,State,var
0,New York,NY,New York%2C+NY
1,Los Angeles,CA,Los Angeles%2C+CA
2,Chicago,IL,Chicago%2C+IL
3,Houston,TX,Houston%2C+TX
4,Phoenix,AZ,Phoenix%2C+AZ
5,Philadelphia,PA,Philadelphia%2C+PA
6,San Antonio,TX,San Antonio%2C+TX
7,San Diego,CA,San Diego%2C+CA
8,Dallas,TX,Dallas%2C+TX
9,San Jose,CA,San Jose%2C+CA


In [5]:
var = pd.Series(test_df['var'])

In [7]:
csv_path = "raw_covid_data/DS-C-19-COUNTIES-1.csv"
cov_counties_1_df  = pd.read_csv(csv_path, encoding="utf-8")

In [8]:
cov_counties_1_df.head()

,FIPS,State,Area_Name,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,POP_ESTIMATE_2018,N_POP_CHG_2018,Births_2018,Deaths_2018,...,Total number of UCR (Uniform Crime Report) Index crimes excluding arson.,Total number of UCR (Uniform Crime Report) index crimes reported including arson,MURDER,RAPE,ROBBERY,Number of AGGRAVATED ASSAULTS,BURGLRY,LARCENY,MOTOR VEHICLE THEFTS,ARSON
0,0,US,United States,NaN,NaN,NaN,327167434,2020313.0,3855500.0,2814013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,AL,Alabama,NaN,NaN,NaN,4887871,12751.0,57216.0,53425.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,AL,Autauga County,2.0,2.0,0.0,55601,158.0,655.0,532.0,...,139.0,1765.0,3.0,15.0,34.0,87.0,447.0,1233.0,85.0,108.0
3,1003,AL,Baldwin County,3.0,2.0,5.0,218022,5403.0,2254.0,2228.0,...,446.0,4988.0,8.0,30.0,76.0,332.0,967.0,3829.0,192.0,31.0
4,1005,AL,Barbour County,6.0,6.0,3.0,24881,-277.0,261.0,324.0,...,48.0,474.0,0.0,4.0,8.0,36.0,90.0,362.0,21.0,0.0


In [9]:
clean_county1_df = cov_counties_1_df[['Area_Name', 'State', 'POP_ESTIMATE_2018', 'Density per square mile of land area - Population', 'Unemployment_rate_2018', 'Median_Household_Income_2018', 'Total_Male', 'Total_Female', 'Total_age0to17', 'Male_age0to17', 'Female_age0to17', 'Total_age18to64', 'Male_age18to64', 'Female_age18to64', 'Total_age65plus', 'Male_age65plus', 'Female_age65plus', 'Total_age85plusr', 'Male_age85plusr', 'Female_age85plusr', 'ICU Beds', 'transit_scores - population weighted averages aggregated from town/city level to county']]
clean_county1_df.head()

,Area_Name,State,POP_ESTIMATE_2018,Density per square mile of land area - Population,Unemployment_rate_2018,Median_Household_Income_2018,Total_Male,Total_Female,Total_age0to17,Male_age0to17,...,Male_age18to64,Female_age18to64,Total_age65plus,Male_age65plus,Female_age65plus,Total_age85plusr,Male_age85plusr,Female_age85plusr,ICU Beds,transit_scores - population weighted averages aggregated from town/city level to county
0,United States,US,327167434,87.4,3.9,61937.0,161128679.0,166038755.0,73399342.0,37490849.0,...,100331012.0,101005887.0,52431193.0,23306818.0,29124375.0,6544503.0,2325693.0,4218810.0,NaN,NaN
1,Alabama,AL,4887871,94.4,3.9,49881.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autauga County,AL,55601,91.8,3.6,59338.0,26995.0,28606.0,13163.0,6726.0,...,16482.0,17303.0,8653.0,3787.0,4866.0,871.0,329.0,542.0,6.0,0.0
3,Baldwin County,AL,218022,114.6,3.6,57588.0,105657.0,112365.0,47110.0,23810.0,...,61166.0,65175.0,44571.0,20681.0,23890.0,4304.0,1755.0,2549.0,51.0,368329051.0
4,Barbour County,AL,24881,31.0,5.2,34382.0,13133.0,11748.0,5192.0,2630.0,...,8414.0,6443.0,4832.0,2089.0,2743.0,472.0,148.0,324.0,5.0,0.0


In [10]:
clean_county1_df.rename(columns={"Area_Name": "County",
                                 "POP_ESTIMATE_2018": "Population_2018",
                                 "Density per square mile of land area - Population": "Population_Density_Per_Square_Mile",
                                 "transit_scores - population weighted averages aggregated from town/city level to county": "Aggregate_Transit_Score",
                                 "ICU Beds": "ICU_Beds"
                                }, inplace=True)
clean_county1_df.head()

C:\Users\jakob\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,County,State,Population_2018,Population_Density_Per_Square_Mile,Unemployment_rate_2018,Median_Household_Income_2018,Total_Male,Total_Female,Total_age0to17,Male_age0to17,...,Male_age18to64,Female_age18to64,Total_age65plus,Male_age65plus,Female_age65plus,Total_age85plusr,Male_age85plusr,Female_age85plusr,ICU_Beds,Aggregate_Transit_Score
0,United States,US,327167434,87.4,3.9,61937.0,161128679.0,166038755.0,73399342.0,37490849.0,...,100331012.0,101005887.0,52431193.0,23306818.0,29124375.0,6544503.0,2325693.0,4218810.0,NaN,NaN
1,Alabama,AL,4887871,94.4,3.9,49881.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autauga County,AL,55601,91.8,3.6,59338.0,26995.0,28606.0,13163.0,6726.0,...,16482.0,17303.0,8653.0,3787.0,4866.0,871.0,329.0,542.0,6.0,0.0
3,Baldwin County,AL,218022,114.6,3.6,57588.0,105657.0,112365.0,47110.0,23810.0,...,61166.0,65175.0,44571.0,20681.0,23890.0,4304.0,1755.0,2549.0,51.0,368329051.0
4,Barbour County,AL,24881,31.0,5.2,34382.0,13133.0,11748.0,5192.0,2630.0,...,8414.0,6443.0,4832.0,2089.0,2743.0,472.0,148.0,324.0,5.0,0.0


In [53]:
clean_county1_df['County'].replace(' County', '', regex=True, inplace=True)
clean_county1_df.head()

C:\Users\jakob\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,County,State,Population_2018,Population_Density_Per_Square_Mile,Unemployment_rate_2018,Median_Household_Income_2018,Total_Male,Total_Female,Total_age0to17,Male_age0to17,...,Male_age18to64,Female_age18to64,Total_age65plus,Male_age65plus,Female_age65plus,Total_age85plusr,Male_age85plusr,Female_age85plusr,ICU_Beds,Aggregate_Transit_Score
0,United States,US,327167434,87.4,3.9,61937.0,161128679.0,166038755.0,73399342.0,37490849.0,...,100331012.0,101005887.0,52431193.0,23306818.0,29124375.0,6544503.0,2325693.0,4218810.0,NaN,NaN
1,Alabama,AL,4887871,94.4,3.9,49881.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autauga,AL,55601,91.8,3.6,59338.0,26995.0,28606.0,13163.0,6726.0,...,16482.0,17303.0,8653.0,3787.0,4866.0,871.0,329.0,542.0,6.0,0.0
3,Baldwin,AL,218022,114.6,3.6,57588.0,105657.0,112365.0,47110.0,23810.0,...,61166.0,65175.0,44571.0,20681.0,23890.0,4304.0,1755.0,2549.0,51.0,368329051.0
4,Barbour,AL,24881,31.0,5.2,34382.0,13133.0,11748.0,5192.0,2630.0,...,8414.0,6443.0,4832.0,2089.0,2743.0,472.0,148.0,324.0,5.0,0.0


In [12]:
clean_county1_df.to_csv("clean_data/county_data_1.csv", index=False, header=True)

In [14]:
csv_path = "raw_covid_data/DS-C-19-COUNTIES-2.csv"
cov_counties_2_df  = pd.read_csv(csv_path, encoding="utf-8")
cov_counties_2_df.head(30)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
5,2020-01-25,Orange,California,6059.0,1,0
6,2020-01-25,Cook,Illinois,17031.0,1,0
7,2020-01-25,Snohomish,Washington,53061.0,1,0
8,2020-01-26,Maricopa,Arizona,4013.0,1,0
9,2020-01-26,Los Angeles,California,6037.0,1,0


In [44]:
cov_counties2_df = cov_counties_2_df.groupby(['county', 'state']).agg({'cases': 'sum', 'deaths': 'sum'})

In [45]:
cov_counties2_df

cases  deaths
county    state                        
Abbeville South Carolina   1542       0
Acadia    Louisiana       10380     587
Accomack  Virginia        21017     303
Ada       Idaho           40601     930
Adair     Iowa              195       0
...                         ...     ...
Yuma      Arizona         13581     182
          Colorado          564       0
Zapata    Texas             351       0
Zavala    Texas             229       0
Ziebach   South Dakota       26       0

[3012 rows x 2 columns]

In [46]:
cov_counties2_df.reset_index()

,county,state,cases,deaths
0,Abbeville,South Carolina,1542,0
1,Acadia,Louisiana,10380,587
2,Accomack,Virginia,21017,303
3,Ada,Idaho,40601,930
4,Adair,Iowa,195,0
...,...,...,...,...
3007,Yuma,Arizona,13581,182
3008,Yuma,Colorado,564,0
3009,Zapata,Texas,351,0
3010,Zavala,Texas,229,0


In [47]:
cov_counties2_df.index = cov_counties2_df.index.set_names(['County', 'State'])
cov_counties2_df

cases  deaths
County    State                        
Abbeville South Carolina   1542       0
Acadia    Louisiana       10380     587
Accomack  Virginia        21017     303
Ada       Idaho           40601     930
Adair     Iowa              195       0
...                         ...     ...
Yuma      Arizona         13581     182
          Colorado          564       0
Zapata    Texas             351       0
Zavala    Texas             229       0
Ziebach   South Dakota       26       0

[3012 rows x 2 columns]

In [48]:
cov_counties2_df.reset_index(level=['County', 'State'], inplace=True)
cov_counties2_df

,County,State,cases,deaths
0,Abbeville,South Carolina,1542,0
1,Acadia,Louisiana,10380,587
2,Accomack,Virginia,21017,303
3,Ada,Idaho,40601,930
4,Adair,Iowa,195,0
...,...,...,...,...
3007,Yuma,Arizona,13581,182
3008,Yuma,Colorado,564,0
3009,Zapata,Texas,351,0
3010,Zavala,Texas,229,0


In [49]:
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

cov_counties2_df['State'] = cov_counties2_df['State'].map(us_state_abbrev).fillna(cov_counties2_df['State'])
cov_counties2_df

,County,State,cases,deaths
0,Abbeville,SC,1542,0
1,Acadia,LA,10380,587
2,Accomack,VA,21017,303
3,Ada,ID,40601,930
4,Adair,IA,195,0
...,...,...,...,...
3007,Yuma,AZ,13581,182
3008,Yuma,CO,564,0
3009,Zapata,TX,351,0
3010,Zavala,TX,229,0


In [50]:
cov_counties2_df.to_csv("clean_data/county_data_2.csv", index=False, header=True)

In [55]:
cov_counties_merge = pd.merge(clean_county1_df, cov_counties2_df, how='outer', on=['County', 'State'])

cov_counties_merge

,County,State,Population_2018,Population_Density_Per_Square_Mile,Unemployment_rate_2018,Median_Household_Income_2018,Total_Male,Total_Female,Total_age0to17,Male_age0to17,...,Total_age65plus,Male_age65plus,Female_age65plus,Total_age85plusr,Male_age85plusr,Female_age85plusr,ICU_Beds,Aggregate_Transit_Score,cases,deaths
0,United States,US,327167434.0,87.4,3.9,61937.0,161128679.0,166038755.0,73399342.0,37490849.0,...,52431193.0,23306818.0,29124375.0,6544503.0,2325693.0,4218810.0,NaN,NaN,NaN,NaN
1,Alabama,AL,4887871.0,94.4,3.9,49881.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autauga,AL,55601.0,91.8,3.6,59338.0,26995.0,28606.0,13163.0,6726.0,...,8653.0,3787.0,4866.0,871.0,329.0,542.0,6.0,0.0,4434.0,152.0
3,Baldwin,AL,218022.0,114.6,3.6,57588.0,105657.0,112365.0,47110.0,23810.0,...,44571.0,20681.0,23890.0,4304.0,1755.0,2549.0,51.0,368329051.0,10465.0,278.0
4,Barbour,AL,24881.0,31.0,5.2,34382.0,13133.0,11748.0,5192.0,2630.0,...,4832.0,2089.0,2743.0,472.0,148.0,324.0,5.0,0.0,3157.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3382,Webster,LA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5594.0,254.0
3383,West Baton Rouge,LA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6512.0,1161.0
3384,West Carroll,LA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,719.0,0.0
3385,West Feliciana,LA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7482.0,261.0


In [54]:
cov_counties_merge.to_csv("clean_data/county_data_merged.csv", index=False, header=True)